# Aironetの設定

## 接続を確認する

工場出荷時設定の[Aironet1140](https://www.cisco.com/c/ja_jp/td/docs/nma/works/works4windows/gs/001/ap1140aut-getstart.html)を2台、wifiのネットワークに繋ぐ。vyosの設定に、macアドレスに対して固定のIPアドレスを払い出すように書いてあるので、Aironetがそのアドレスを掴めたか確認する。起動にそこそこ時間がかかるので、時間を空けて何度か試すこと。

因みにAironet1140を工場出荷時設定に戻すには、「MODE」スイッチを押したまま電源を疎通させランプが赤点灯になるまで押し続けることで行える。

In [ ]:
!ping -c 4 ap64
!test $(ping -c 4 ap64 | grep "4 packets received" | wc -l) -eq 1
!ping -c 4 ap83
!test $(ping -c 4 ap83 | grep "4 packets received" | wc -l) -eq 1

## Ansibleでssh接続を設定する

工場出荷時設定のAironetは、telnetでデフォルトアカウントで入れるようになっているので、新規のアカウントで、sshでしか入れないように設定する。

[playbook](/edit/wifi/aironet-setup.yml)

In [ ]:
!ansible-playbook -i ap64,ap83, aironet-setup.yml -t ssh-setup -vvv

## fingerprintを記録する

ansibleがAironetにsshするためにknown_hostsにfingerprintを覚えさせる必要がある。ansibleの設定で自動登録できるらしいが、jupyter内に入れてあるansibleのバージョンの問題か、動作しなかったので、ssh-keyscanで対応する。Aironetへのssh接続でfingerprintの確認が出ない状態にできたら、アクセスポイントの設定を行える。

In [ ]:
!mkdir -p ~/.ssh
!ssh-keyscan -H ap64 > ~/.ssh/known_hosts
!ssh-keyscan -H ap83 >> ~/.ssh/known_hosts

## AnsibleでWiFiを設定する

In [ ]:
!ansible-playbook -i ap64,ap83, aironet-setup.yml -t setup -vvv